In [1]:
# Face Detection vs Face Recognition

import cv2
import mediapipe as mp
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

'''
# For static images:
IMAGE_FILES = []
with mp_face_detection.FaceDetection(
    model_selection=1, min_detection_confidence=0.5) as face_detection:
  for idx, file in enumerate(IMAGE_FILES):
    image = cv2.imread(file)
    # Convert the BGR image to RGB and process it with MediaPipe Face Detection.
    results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Draw face detections of each face.
    if not results.detections:
      continue
    annotated_image = image.copy()
    for detection in results.detections:
      print('Nose tip:')
      print(mp_face_detection.get_key_point(
          detection, mp_face_detection.FaceKeyPoint.NOSE_TIP))
      mp_drawing.draw_detection(annotated_image, detection)
    cv2.imwrite('/tmp/annotated_image' + str(idx) + '.png', annotated_image)
'''

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.5) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
          # If loading a video, use 'break' instead of 'continue'.
            continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.detections:
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection)
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection', cv2.flip(image, 1)) # flip 좌우바전
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()

In [20]:
import cv2
import mediapipe as mp


def overlay(image, x, y, w, h, overlay_image): # 3채널 -> 4채널
    alpha = overlay_image[:,:,3]
    mask_image = alpha / 255  # 0~1 사이값 1: 불투명, 0: 투명
    #(255,255) -> (1,1)
    #(255, 0) -> (1,0)
    #1-mase_image 
    # (0,0)
    #(1,0) 반전된 영역의 값이됨    투명영역은 배경에서 불투명 영역은 이미지에서
    
    for c in range(0,3): 
        image[y-h:y+h, x-w:x+w, c] = (overlay_image[:,:,c] * mask_image) + (image[y-h:y+h, x-w:x+w, c] * (1-mask_image))

#얼굴을 찾고 얼굴에 표시하기위해
mp_face_detection = mp.solutions.face_detection  # 얼굴 검출을 위한 face_detection 
mp_drawing = mp.solutions.drawing_utils  #  얼굴의 특징을 그리기 위한 drawing_utils 모듈 사용

# For webcam input:
cap = cv2.VideoCapture('face_video.mp4')
    
# 이미지 불러오기 
image_right_eye = cv2.imread('RR_pig.png', cv2.IMREAD_UNCHANGED)
image_left_eye =  cv2.imread('LL_pig.png', cv2.IMREAD_UNCHANGED)
image_nose = cv2.imread('nose_pig.png', cv2.IMREAD_UNCHANGED)

#
# model_selection = 0 (가까운 사람) or 1 (먼사람)
# min_detection_confidence 0.5 (50 퍼센트 정도 확실하면 사람 얼굴)
#
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections:
            # 특징 오른쪽 눈, 왼쪽 눈, 코, 입중심, 오른쪽 귀, 왼쪽 귀
            for detection in results.detections:
                #mp_drawing.draw_detection(image, detection)
                #print(detection)
                
                #특징 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]
                left_eye = keypoints[1]
                nose_tip = keypoints[2]
                
                h, w, _ = image.shape # height, width, channel : 이미지로부터 세로, 가로, 크기 가져옴
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100)
                left_eye = (int(left_eye.x * w) + 20, int(left_eye.y * h) - 100)
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                # 양 눈 동그라미 그리기
                #cv2.circle(image, right_eye, 50, (255,0,0), 10, cv2.LINE_AA)
                #cv2.circle(image, left_eye, 50, (0,255,0), 10, cv2.LINE_AA)
                #cv2.circle(image, nose_tip, 50, (0,255,255), 10, cv2.LINE_AA)
                
                #각 특징 이미지 그리기
                #image[right_eye[1] - 50:right_eye[1] + 50, right_eye[0] - 50: right_eye[0] + 50] = image_right_eye
                #image[left_eye[1] - 50:left_eye[1] + 50, left_eye[0] - 50: left_eye[0] + 50] = image_left_eye
                #image[nose_tip[1] - 70:nose_tip[1] + 70, nose_tip[0] - 70: nose_tip[0] + 70] = image_nose
                
                overlay(image, *right_eye, 50, 50, image_right_eye)
                overlay(image, *left_eye, 50, 50, image_left_eye)
                overlay(image, *nose_tip, 70, 70, image_nose)
                
            
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection',cv2.resize(image, None, fx =0.5, fy = 0.5))
        
        if cv2.waitKey(1) == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()